## Descriptors

- [**Descriptors**](#descriptors)
- [**Getters and Setters**](#getters_and_setters)
- [**Strong and Weak References**](#strong_and_weak_references)
- [**Using as Instance Properties**](#using_as_instance_properties)
- [**The \_\_set\_name\_\_ Method**](#the_set_name_method)
- [**Property Lookup Resolution**](#property_lookup_resolution)
- [**Properties and Descriptors**](#properties_and_descriptors)
- [**Functions and Descriptors**](#functions_and_descriptors)

---

### Descriptors <a name='descriptors'></a>

Descriptors are Python objects that implement at least one of the methods of `descriptor protocol`:

* \_\_get\_\_(self, obj, type=None) -> object
* \_\_set\_\_(self, obj, value) -> None
* \_\_delete\_\_(self, obj) -> None
* \_\_set_name\_\_(self, owner, name)

And they can be used to control accessing properties.

* For those that implement `__get__` only -> non-data descriptors
* For those that implement `__set__` or `__delete__` -> data descriptors

> Example 1: Show that descriptors can be called on both class and instance level

In [1]:
from datetime import datetime

In [2]:
class UtcNow:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

In [3]:
class Logger:
    current_time = UtcNow()

In [4]:
print('Class level: ', Logger.current_time)

Class level:  2023-07-13T15:06:02.844567


In [5]:
l = Logger()
print('Instance level: ', l.current_time)

Instance level:  2023-07-13T15:06:02.861561


> Example 2: Show that descriptor can be used to create a more generic way of controlling passed-in objects

In [6]:
from random import choice, seed

In [7]:
class Deck:
    @property
    def suit(self):
        return choice(('Spade', 'Heart', 'Diamond', 'Club'))
    
    @property
    def card(self):
        return choice(tuple('23456789JOKA') + ('10', ))

In [8]:
d = Deck()

In [9]:
seed(66)
for _ in range(10):
    print(d.suit, d.card)

Spade 6
Club 5
Club 6
Diamond 10
Spade A
Club 4
Club J
Heart 10
Spade A
Diamond A


In [10]:
class Choice():
    def __init__(self, *choices):
        self.choices = choices
        
    def __get__(self, instance, owner_class):
        return choice(self.choices) # Generic way of randomly choosing any object passed in

In [11]:
class DeckWithDescriptor():
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JOKA' + '10')

In [12]:
d = DeckWithDescriptor()

In [13]:
seed(66)
for _ in range(10):
    print(d.suit, d.card)

Spade 6
Club 5
Club 6
Diamond 0
Spade A
Club 4
Club J
Heart 1
Spade A
Diamond A


---

### Getters and Setters <a name='getters_and_setters'></a>

> `__get__` method:  
Normally different values will be returned from \_\_get\_\_ method depending on:
> * called from class (return the descriptor instance)
> * called from instance (return the attribute value)
> 
> Hence the signature of \_\_get\_\_ method is:  
> ```Python
def __get__(self, instance, owner_class)
```

In [14]:
# Self-created descriptor
class UtcNow:
    def __get__(self, instance, owner_class):
        print(f'__get__ method called, self={self}, instance={instance}, owner_class={owner_class}')
        return datetime.utcnow().isoformat()

In [15]:
class Logger1:
    current_time = UtcNow()

In [16]:
class Logger2:
    current_time = UtcNow()

In [17]:
# instance is None when calling from class level
Logger1.current_time

__get__ method called, self=<__main__.UtcNow object at 0x000001BF03581BB0>, instance=None, owner_class=<class '__main__.Logger1'>


'2023-07-13T15:06:03.050021'

In [18]:
Logger2.current_time

__get__ method called, self=<__main__.UtcNow object at 0x000001BF03581B80>, instance=None, owner_class=<class '__main__.Logger2'>


'2023-07-13T15:06:03.065797'

In [19]:
l1 = Logger1()
l1.current_time

__get__ method called, self=<__main__.UtcNow object at 0x000001BF03581BB0>, instance=<__main__.Logger1 object at 0x000001BF0358A6D0>, owner_class=<class '__main__.Logger1'>


'2023-07-13T15:06:03.081647'

In [20]:
l2 = Logger2()
l2.current_time

__get__ method called, self=<__main__.UtcNow object at 0x000001BF03581B80>, instance=<__main__.Logger2 object at 0x000001BF0358ADF0>, owner_class=<class '__main__.Logger2'>


'2023-07-13T15:06:03.100079'

> `__set__` method:  
> \_\_set\_\_ method is always called from instances, thus its signature is:
> ```Python 
def __set__(self, instance, value)
```

__Caveat__: 
When referencing the same object in descriptor, the instances created from the class that uses descriptor will inter-overwrite each other's value when calling \_\_set\_\_ method. Therefore we would need a way to make the changes to attributes instance-specific.

In [21]:
class IntegerValue:
    def __set__(self, instance, value):
        self._value = value
        print(f'__set__ method called, instance={instance}, value={value}')
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ method called from class')
        else:
            print(f'__get__ method called, instance={instance}, owner_class={owner_class}')
            return self._value

In [22]:
class Point:
    x = IntegerValue()
    y = IntegerValue()

In [23]:
Point.x

__get__ method called from class


In [24]:
p1 = Point()
p2 = Point()

In [25]:
p1.x = 10

__set__ method called, instance=<__main__.Point object at 0x000001BF0346BBE0>, value=10


In [26]:
p2.x = 5

__set__ method called, instance=<__main__.Point object at 0x000001BF034703A0>, value=5


In [27]:
# Attribute of p1 is overwritten when p2 changes its attribute
p1.x

__get__ method called, instance=<__main__.Point object at 0x000001BF0346BBE0>, owner_class=<class '__main__.Point'>


5

---

### Strong and Weak References <a name='strong_and_weak_references'></a>

> `Strong reference`:
> 
> When create an instance of one object and make a new instance pointing to the same object as:
> ```Python
p1 = Person()
p2 = p1
> ```
> p1 and p2 both have strong references to the object.

In [28]:
import ctypes

In [29]:
def ref_count(address):
    # Cound references of an object's id
    return ctypes.c_long.from_address(address).value

In [30]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f'Person (name={self.name})'

In [31]:
p1 = Person('Taylor')
p2 = p1
print(f'Object reference count is: {ref_count(id(p1))}')

Object reference count is: 2


In [32]:
# Deleting one reference will not influence other strong references
del p1
print(f'Object reference count is: {ref_count(id(p2))}')

Object reference count is: 1


> `Weak reference`:
> 
> Weak reference, in contrast, is a reference to an object that does not affect the reference count as far as the memory manager is concerned. Therefore, deleting the original instance pointing to the object will also make the other instances weakly pointing to it "dead".
> 
> __Note__: Weak reference is not available for most built-in types such as list, dict.

In [33]:
import weakref

In [34]:
person1 = Person('Tay')
person2 = weakref.ref(person1)

In [35]:
print(f'Strong reference count is: {ref_count(id(person1))}')
print(f'Weak reference count is: {weakref.getweakrefcount(p1)}')

Strong reference count is: 1


NameError: name 'p1' is not defined

In [36]:
# weakref is callable to the object that it's pointing to
person2() is person1

True

In [37]:
del person1
print(person2)

<weakref at 0x000001BF03466950; dead>


---

### Using as Instance Properties <a name='using_as_instance_properties'></a>

In [38]:
import weakref

In [39]:
class IntegerValue:
    def __init__(self):
#         self.values = {} # infeasible approach results in potential memory leak
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values[instance]

In [40]:
class Point:
    x = IntegerValue()

In [41]:
p = Point()
p.x = 100.1
print(p.x)
print(Point.x.values.keyrefs())

100
[<weakref at 0x000001BF03578310; to 'Point' at 0x000001BF03441B20>]


In [42]:
# Week references will be cleaned up automatically when strong ref is removed
del p
print(Point.x.values.keyrefs())

[]


---

### The \_\_set_name\_\_ Method <a name='the_set_name_method'></a>

\_\_set_name\_\_ method shows the name of variables that has been called, useful for logging purpose.

In [43]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
    
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner={owner_class}, property_name={property_name}')
        self.property_name = property_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be string type.')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be at least {self.min_length} chars.')   
        instance.__dict__[self.property_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print(f'__get__ called for property {self.property_name} of instance {instance}')
        return instance.__dict__.get(self.property_name, None)

In [44]:
class Person:
    
    first_name = ValidString(1)
    last_name = ValidString(2)

__set_name__: owner=<class '__main__.Person'>, property_name=first_name
__set_name__: owner=<class '__main__.Person'>, property_name=last_name


In [45]:
p = Person()
try:
    p.first_name = 'Taylor'
    p.last_name = 'H'
except ValueError as ex:
    print(ex)

last_name must be at least 2 chars.


---

### Property Lookup Resolution <a name='property_lookup_resolution'></a>

For a class that has a descriptor and an instance dictionary, calling obj.x will use `__dict__` entry or `descriptor` in different cases:

* For data descriptors:  
Calling obj.x will always call descriptor and override the instance dictionary, so editing \_\_dict\_\_ directly will not influence the property value.

In [46]:
class IntegerValue:
    def __set__(self, instance, value):
        print('__set__ called...')
    
    def __get__(self, instance, owner_class):
        print('__get__ called...')

In [47]:
class Point:
    x = IntegerValue()

In [48]:
p = Point()

In [49]:
p.x = 100

__set__ called...


In [50]:
p.x

__get__ called...


In [51]:
# Edit property value in __dict__ directly
p.__dict__['x'] = 'hello'
p.__dict__

{'x': 'hello'}

In [52]:
# x in __dict__ will be ignored, and __get__method will be called directly
p.x

__get__ called...


* For non-data descriptors:  
Calling obj.x will look into the instance dictionary first, and uses descriptor if not present (which means it will access \_\_dict\_\_\['x'\] first then try to get the value using \_\_get\_\_ method).

In [53]:
class TimeUtc:
    def __get__(self, instance, owner_class):
        print('__get__ called...')

In [54]:
class Logger:
    current_time = TimeUtc()

In [55]:
l = Logger()

In [56]:
l.current_time

__get__ called...


In [57]:
l.__dict__

{}

In [58]:
l.__dict__['current_time'] = 'hello'

In [59]:
# Look into __dict__ first to see if current_time exists
l.current_time

'hello'

---

### Properties and Descriptors <a name='properties_and_descriptors'></a>

Property is indeed a data descriptor that implements `__get__`, `__set__` and `__delete__`.

In [60]:
class UtcNow:
    @property
    def current_time(self):
        return 'current_time'

In [61]:
print(hasattr(UtcNow.current_time, '__get__'))
print(hasattr(UtcNow.current_time, '__set__'))
print(hasattr(UtcNow.current_time, '__delete__'))

True
True
True


In [62]:
from numbers import Integral

In [63]:
class Person:
    @property
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be a non-negative integer')
        self._age = value

In [64]:
p = Person()

try:
    p.age = -10
except Exception as ex:
    print(ex)

age: must be a non-negative integer


In [65]:
p.age = 10
# Property `age` is not in the __dict__
p.__dict__

{'_age': 10}

---

### Functions and Descriptors <a name='functions_and_descriptors'></a>

Function is indeed an object that implements non-data descriptor protocol.